In [0]:
import numpy as np
import chainer
import chainer.functions as F
import chainer.links as L
import chainer.initializers as I
from chainer import training
from chainer.training import extensions

In [0]:
class MyChain(chainer.Chain):
    def __init__(self):
        super(MyChain, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(2, 3) # 入力2, 出力3
            self.l2 = L.Linear(3, 2)
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        y = self.l2(h1)
        return y

In [0]:
epoch = 100
batchsize = 4

In [0]:
trainx = np.array(([0, 0], [0, 1], [1, 0], [1, 1]), dtype=np.float32)
trainy = np.array([0, 1, 1, 1], dtype=np.int32)
train = chainer.datasets.TupleDataset(trainx, trainy)
test = chainer.datasets.TupleDataset(trainx, trainy)

In [23]:
model = L.Classifier(MyChain(), lossfun=F.softmax_cross_entropy)
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [0]:
train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)

In [0]:
updater = training.StandardUpdater(train_iter, optimizer)

In [0]:
trainer = training.Trainer(updater, (epoch, 'epoch'))

#Extensions

In [0]:
# trainer.extend(extensions.ProgressBar())
trainer.extend(extensions.LogReport()) #ログ出力
trainer.extend(extensions.Evaluator(test_iter, model)) # エポック数の表示
trainer.extend(extensions.PrintReport(entries=['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy', 'elapsed_time' ]))
trainer.extend(extensions.dump_graph('main/loss')) # nnの構造
trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], 'epoch', file_name='loss.png')) # 誤差グラフ
trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'], 'epoch', file_name='accuracy.png')) # 精度グラフ
trainer.extend(extensions.snapshot(), trigger=(100, 'epoch')) # 学習再開のためのスナップショット出力

In [0]:
# chainer.serializers.load_npz('result/snapshot_iter_500', trainer) # 再開用
# chainer.serializers.save_npz('result/out.model', model)

#Train

In [29]:
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           1.30642     1.3007                0.25           0                         0.00597534    
2           1.3007      1.295                 0              0                         0.338233      
3           1.295       1.28931               0              0                         0.675683      
4           1.28931     1.28364               0              0                         1.03855       
5           1.28364     1.27798               0              0                         1.38721       
6           1.27798     1.27234               0              0                         1.74501       
7           1.27234     1.26672               0              0                         2.09706       
8           1.26672     1.26111               0              0                         2.57457       
9           1.26111     1.25552               0              0                      